In [1]:
import enum
import importlib
import inspect
import os
import shutil

from typing import List
from IPython.display import HTML
from sflkit.analysis.spectra import Line

from sflkit.analysis import analysis_type, factory
from sflkit import *
from sflkit.color import ColorCode
from sflkit import instrument_config, analyze_config
from sflkit.config import Config

from avicenna import *

In [2]:
import string
import csv
import logging

from pathlib import Path
from debugging_framework.input.oracle import OracleResult
from isla.solver import ISLaSolver


In [3]:
middle_grammar = {
    "<start>": ["<stmt>"],
    "<stmt>": ["<x>,<y>,<z>"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

In [4]:
# 321, 312, 213
middle_inputs = ['2,3,1', '3,1,2', '2,1,3']

In [5]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'
language='python'
predicates='line'
metrics='Tarantula'
#passing='event-files/0,event-files/1'
failing='event-files/2'
def get_config():
    return Config.create(path=middle_py,
                         working=tmp_py,
                         language=language,
                         predicates=predicates,
                         metrics=metrics,
                         failing=failing)
def instrument(out=True):
    instrument_config(get_config())
    if out:
        with open(tmp_py, 'r') as fp:
            print(fp.read())

In [6]:
instrument()

sflkit :: INFO     :: I found 11 events in middle.py.
sflkit :: INFO     :: I found 11 events in middle.py.


import sflkitlib.lib

def middle(x, y, z):
    sflkitlib.lib.add_line_event(0)
    m = z
    sflkitlib.lib.add_line_event(1)
    if y < z:
        sflkitlib.lib.add_line_event(2)
        if x < y:
            sflkitlib.lib.add_line_event(3)
            m = y
        else:
            sflkitlib.lib.add_line_event(4)
            if x < z:
                sflkitlib.lib.add_line_event(5)
                m = y
    else:
        sflkitlib.lib.add_line_event(6)
        if x > y:
            sflkitlib.lib.add_line_event(7)
            m = y
        else:
            sflkitlib.lib.add_line_event(8)
            if x > z:
                sflkitlib.lib.add_line_event(9)
                m = x
    sflkitlib.lib.add_line_event(10)
    return m


In [7]:
def analyzer_conf(conf: Config, factory):
    analyzer = Analyzer(irrelevant_event_files=conf.failing, 
                        relevant_event_files=conf.passing,
                        factory=factory)
    return analyzer

In [8]:
factory = factory.LineFactory()
line_event = analysis_type.AnalysisType(0)
line_event

LINE

In [9]:
analyzer = analyzer_conf(get_config(), factory=factory)

In [10]:
# results = analyzer.analyze()
# results

# analyzer needs event files 
analyzer.analyze()

In [11]:
coverage: List[Line] = analyzer.get_coverage()
coverage = {line.line for line in coverage}
print(coverage)
print(type(coverage.pop()))
print(type(coverage.pop()))
print(coverage)
print(type(coverage.pop()))
print(type(coverage.pop()))
print(type(coverage.pop()))
print(coverage)
print(type(3))
if 3 in coverage:
    print("is doch")

{2, 3, 9, 10, 13}
<class 'int'>
<class 'int'>
{9, 10, 13}
<class 'int'>
<class 'int'>
<class 'int'>
set()
<class 'int'>


In [12]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [13]:
from importlib import resources

def get_avicenna_rsc_path() -> Path:
    with resources.path('avicenna', 'rsc') as p:
        return Path(p)

In [14]:
path = str(get_avicenna_rsc_path() )+ '/instrumented.py'
print(path)

/home/lina/ba/avicenna/src/avicenna/rsc/instrumented.py


In [15]:
from avicenna.avix import *
from avicenna.oracle_construction import * 
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            line = 4,
                            put_path='middle.py'
                            )


In [16]:

avix_oracle('2,5,3')

<OracleResult.PASSING: 'PASSING'>

In [17]:
avix_oracle('2,1,3')

<OracleResult.FAILING: 'FAILING'>

In [18]:
def middle_oracle(x, y, z):
    # convert inputs
    
    sorted_inputs = [x, y, z]
    sorted_inputs.sort()
    return sorted_inputs[1]

In [19]:
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        else:
            if x < z:
                m = y
    else:
        if x > y:
            m = y
        else:
            if x > z:
                m = x
    return m

In [20]:
# AviX.instrument_avix(put_path='middle.py',
#                     instr_path='../src/avicenna/rsc/instrumented.py' )

In [21]:
regular_oracle = construct_oracle(
                            program_under_test=middle,
                            timeout=10,
                            program_oracle=middle_oracle,
                            default_oracle_result=OracleResult.PASSING)



In [22]:
param = list(map(int, str('2,1,3').strip().split(',')))
middle_inputs

['2,3,1', '3,1,2', '2,1,3']

In [23]:
regular_oracle('3,1,2')


<OracleResult.PASSING: 'PASSING'>

In [24]:
# avicenna = Avicenna(
#     grammar=middle_grammar,
#     initial_inputs=middle_inputs,
#     oracle=avix_oracle,
#     max_iterations=10,
#     top_n_relevant_features=3,
#     #max_conjunction_size=6,
# )

In [25]:
avix = AviX(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=avix_oracle,
    max_iterations=10,
    top_n_relevant_features=3,
    put_path='middle.py',
    
    #max_conjunction_size=6,
)

sflkit :: INFO     :: I found 11 events in middle.py.
sflkit :: INFO     :: I found 11 events in middle.py.


In [26]:
avicenna2 = Avicenna(
    grammar=middle_grammar,
    initial_inputs=middle_inputs,
    oracle=regular_oracle,
    max_iterations=10,
    top_n_relevant_features=3, 
)

In [27]:
avix.patterns

[ForallFormula(NonterminalPlaceholderVariable(name='elem', n_type='<?NONTERMINAL>'), Constant("start", "<start>"), SMTFormula('(>= (str.to_int elem) (str.to_int STRING_0))', StringPlaceholderVariable(name='STRING_0'), NonterminalPlaceholderVariable(name='elem', n_type='<?NONTERMINAL>'), )),
 ExistsFormula(NonterminalPlaceholderVariable(name='container', n_type='<?NONTERMINAL>'), Constant("start", "<start>"), ExistsFormula(NonterminalPlaceholderVariable(name='length_field', n_type='<?NONTERMINAL>'), Constant("start", "<start>"), SMTFormula('(<= (str.len container) (str.to_int length_field))', NonterminalPlaceholderVariable(name='length_field', n_type='<?NONTERMINAL>'), NonterminalPlaceholderVariable(name='container', n_type='<?NONTERMINAL>'), ))),
 ForallFormula(NonterminalPlaceholderVariable(name='elem', n_type='<?NONTERMINAL>'), Constant("start", "<start>"), SMTFormula('(<= (str.to_int elem) (str.to_int STRING_0))', StringPlaceholderVariable(name='STRING_0'), NonterminalPlaceholderVar

In [28]:
import warnings
# Suppress the specific SHAP warning
warnings.filterwarnings(
    "ignore",
    message="LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray",
)
warnings.filterwarnings(
    "ignore", 
    message="No further splits with positive gain, best gain: -inf"
)

In [29]:
logging.basicConfig(filename='avix.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
# only 2 constraints used in the end why
best_invariant = avix.explain() # unparse with islaunparse for further use

In [30]:
logging.basicConfig(filename='avicenna.log', filemode='w', encoding='utf-8', level=logging.INFO, force=True)
#another_invariant = avicenna2.explain() # unparse with islaunparse for further use

In [31]:
#print(another_invariant)

In [47]:
print(best_invariant)
print(best_invariant[0])
constraint = best_invariant[0]

(ForallFormula(BoundVariable("elem_1", "<z>"), Constant("start", "<start>"), ExistsFormula(BoundVariable("elem_2", "<y>"), Constant("start", "<start>"), SMTFormula('(> (str.to_int elem_1) (str.to_int elem_2))', BoundVariable("elem_1", "<z>"), BoundVariable("elem_2", "<y>"), ))), 1.0, 1.0)
∀ elem_1 ∈ start: (∃ elem_2 ∈ start: (StrToInt(elem_1) > StrToInt(elem_2)))


In [48]:
# solver1 = ISLaSolver(
#     grammar=middle_grammar,
#     formula=another_invariant[0],
#     enable_optimized_z3_queries=False,
# )

solver2 = ISLaSolver(
    grammar=middle_grammar,
    formula=constraint,
    enable_optimized_z3_queries=False,
)

results1 = []
results2 = []

# # print(best_invariant[0])
# # print(another_invariant[0])
# for _ in range(1,10):
#     results1.append(solver1.solve())
    
print("stop")
#    avix_oracle()
for _ in range(1,10):
    results2.append(solver2.solve())

stop


In [50]:
for item in results2:
    print(item)

1,1,2
378,1,2
5,1,2
2694,1,2
969,1,2
6,1,2
33,1,2
6,1,2
34,1,2


In [34]:
for input in results1:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))

In [35]:
for input in results2:
    if avix_oracle(input) == OracleResult.FAILING:
        print(input)
        print(avix_oracle(input))
for input in results2:
    print(input)

94,2,7
FAILING
6,2,7
FAILING
8,2,7
FAILING
3,2,7
FAILING
5,2,7
FAILING
21,2,7
FAILING
735,2,7
FAILING
88,2,7
FAILING
841,2,7
FAILING
94,2,7
6,2,7
8,2,7
3,2,7
5,2,7
21,2,7
735,2,7
88,2,7
841,2,7


In [36]:
solver = ISLaSolver(
    grammar=middle_grammar,
    formula=best_invariant[0],
    enable_optimized_z3_queries=False,
)

# this isnt working rn why not raaaaaaaaaaaaaaaaaaaaaaa
# should be inputs of type 2, 3, 1
for _ in range(1,10):
    print(solver.solve())


46,8,9
9,8,9
835,8,9
21,8,9
73,8,9
35,8,9
51934,8,9
612,8,9
3,8,9


In [37]:
# call func for middle, converts string input to usable integer values

def call_func_middle(inp: str):
    
    inp = inp.__str__()
    
    middle_input = inp.split(',')
    converted_inp =  [int(middle_input[0]), int(middle_input[1]), int(middle_input[2])]
    return converted_inp

In [38]:
test_path = Path('rsc/')
print(Path.exists(test_path))


True


In [39]:
from avicenna.avix import *
from avicenna.oracle_construction import * 

In [40]:
def middle_inp_conv(inp):
    inp = inp.__str__()
    middle_input = inp.split(',')
    
    converted_inp = [
        int(middle_input[0]),
        int(middle_input[1]),
        int(middle_input[2])
    ]
    
    return converted_inp

In [41]:
import tmp
importlib.reload(tmp)
tmp.sflkitlib.lib.reset()
avix_oracle = construct_oracle(
                            program_under_test='middle',
                            inp_converter=middle_inp_conv,
                            timeout=10,
                            line = 7,
                            resource_path='rsc/',
                            program_oracle= None)


In [42]:
avix_oracle("6,3,7")

TypeError: option values must be strings

In [ ]:
# # double check if this works

# AviX.create_event_file(instrumented_function='middle',
#                        #instr_path='tmp',
#                        inp = '2,1,3',
#                        conversion_func=middle_inp_conv,
#                        event_path='rsc/event_file'
#                        )

In [ ]:
avix = AviX(grammar=middle_grammar,
            initial_inputs=middle_inputs,
            oracle=avix_oracle,
            max_iterations=10,
            desired_line=7,
            put_path='middle.py',
            #instr_path='instrumented.py',
            min_precision = 0.7,)

TypeError: AviX.__init__() got an unexpected keyword argument 'desired_line'

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer, DerivationTree

middle_grammar_converted = {
    "<start>": ["<stmt>"],
    "<stmt>": ["str.to.int(<x>),str.to.int(<y>),str.to.int(<z>)"],
    "<x>": ["<integer>"],
    "<y>": ["<integer>"],
    "<z>": ["<integer>"],
    "<integer>": ["<digit>", "<digit><integer>"],
    "<digit>": [str(num) for num in range(1, 10)]
}

fuzzer = GrammarFuzzer(middle_grammar)

for i in range(10):
    print(fuzzer.fuzz())

34146,7,89
3925,5,15
5985,6,1
3,4,5179
6,52,714477
2,6,8
78,2,486
114,496,2
71,8,298
76,7,2


https://github.com/uds-se/sflkit

<img src="qrcode.png" style="width:500px">